# Section3：GRU

LSTMの課題

- 課題 \
LSTMでは、パラメータ数が多く、計算負荷が高くなる問題があった。 \
　⇒GRU
 
- GRUとは？ \
従来のLSTMでは、パラメータが多数存在していたため、計算負荷が大きかった。しかし、GRUでは、そのパラメータを大幅に削減し、精度は同等またはそれ以上が望める様になった構造。

- メリット \
計算負荷が低い。

### ゼロから作るディープラーニング２　付録C　GRU

# 実装演習

# 確認テスト

#### P88 
問） \
LSTMとCECが抱える課題について、それぞれ簡潔に述べよ。

#### P92
問） \
LSTMとGRUの違いを簡潔に述べよ。

# Section4：双方向RNN

過去の情報だけでなく、未来の情報を加味することで、精度を向上させるためのモデル

実用例：文章の推敲や、機械翻訳等

# 実装演習